## set running directory to project source

In [1]:
import os
import sys
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

from pkg.utils.code_scraps import add_indy_sweep_tool, \
        use_current_place_point_only, use_current_sub_binders_only, \
        finish_L_shape, set_l_shape_object, ModeSwitcher, double_sweep_motions

from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene
from pkg.planning.pipeline import PlanningPipeline
from pkg.ui.ui_broker import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask, AbstractObject
from pkg.planning.constraint.constraint_subject import SweepLineTask
from pkg.planning.constraint.constraint_subject import SweepFrame
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT
from pkg.planning.constraint.constraint_common import sample_redundancy, combine_redundancy
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
import matplotlib.pyplot as plt
import math
import numpy as np
from collections import defaultdict
from demo_utils.area_select import DATASET_DIR, SweepDirections
try_mkdir(DATASET_DIR)

## create scene builder

In [2]:
ROBOT_TYPE = RobotType.indy7
INDY_BASE_OFFSET = (0.172,0,0.439)
TIMEOUT_MP = 0.5

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                None)
robot_config = RobotConfig(1, ROBOT_TYPE, (INDY_BASE_OFFSET, (0,0,0)),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True})

crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config], connection_list=[False, False])

ROBOT_NAME = robot_config.get_indexed_name()
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

connection command:
kmb0: False
indy1: False


## make geometry scene

In [3]:
BASE_LINK = "base_link"
EE_NAME = "ee_point"
gscene = s_builder.create_gscene(crob, start_rviz=True)

floor = gscene.create_safe(GEOTYPE.BOX, "floor", BASE_LINK, (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
# ee_point.draw_traj_coords([[0]*6])

Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


## init planning pipeline

In [4]:
from demo_utils.environment import add_kiro_indytool_down

pscene = PlanningScene(gscene, combined_robot=crob)
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
add_kiro_indytool_down(gscene, zoff=0, tool_link=TIP_LINK, face_name="dummy_brush")
ee_point = gscene.create_safe(GEOTYPE.SPHERE, EE_NAME, TIP_LINK, (0.01,)*3, 
                              center=(0,0,0), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.8), display=True, fixed=True, collision=False)

ppline = PlanningPipeline(pscene)

mplan = MoveitPlanner(pscene, enable_dual=False, incremental_constraint_motion=True)
mplan.update_gscene()
ppline.set_motion_planner(mplan)


# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


## Set Parameters

In [5]:
TIMEOUT_MP = 0.5

X_step = 0.02
Y_step = 0.02
Z_step = 0.02

VISUALIZE = False
VERBOSE = False
N_try = 3
SWEEP_DIV = 10


HOME_POSE_EXT = np.array([0,]*6 + np.deg2rad([  0., 50.,  -70.,  -0.,  -90., 0]).tolist())
crob.home_pose = HOME_POSE_EXT
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
gscene.show_pose(crob.home_pose)

# # dummy tool coord
# gscene.add_highlight_axis("tool", "coord", link_name="indy0_tcp", 
#                           center=(0.1,0,0.2), orientation_mat=Rot_rpy((0,0,np.pi)))

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Scan

In [ ]:
# HOME_POSE = np.zeros(gscene.joint_num)
HOME_POSE = HOME_POSE_EXT
for sweep_dir in SweepDirections:

    if sweep_dir == SweepDirections.front:
        X_range = (0.3, 1)
        Y_range = (-0.5, 0.5)
        Z_range = (0.0, 1.0)
        SWEEP_AXIS = "X"
    elif sweep_dir == SweepDirections.up:
        X_range = (0.2, 0.8)
        Y_range = (-0.5, 0.5)
        Z_range = (-0.1, 1.3)
        SWEEP_AXIS = "Z"
    elif sweep_dir == SweepDirections.down:
        X_range = (0.2, 0.8)
        Y_range = (-0.5, 0.5)
        Z_range = (-0.7, 0.7)
        SWEEP_AXIS = "Z"
    else:
        raise(NotImplementedError("SweepDirection not implemented"))
        
    DATASET_PATH = os.path.join(DATASET_DIR, SweepDirections.get_file_name(ROBOT_TYPE, sweep_dir.name+SWEEP_AXIS))
    R_be = SweepDirections.get_dcm_re(sweep_dir)
        
    gtimer = GlobalTimer.instance()
    gtimer.reset()

    X_DIR, Y_DIR, Z_DIR = np.identity(3, dtype=np.int)
    SWEEP_AX_IDX = "xyz".find(SWEEP_AXIS.lower())
    SWEEP_DIR = [X_DIR, Y_DIR, Z_DIR][SWEEP_AX_IDX]
    RANGE_STEPS_REF = [(X_range, X_step, X_DIR), (Y_range, Y_step, Y_DIR), (Z_range, Z_step, Z_DIR)]
    RANGE_STEPS = np.array(RANGE_STEPS_REF)[np.where(np.logical_not(SWEEP_DIR))]
    RANGE1, STEP1, DIR1 = RANGE_STEPS[0]
    RANGE2, STEP2, DIR2 = RANGE_STEPS[1]
    RANGEX, STEPX, DIRX = RANGE_STEPS_REF[SWEEP_AX_IDX]
    STEP_SWEEP = float(RANGEX[1] - RANGEX[0])/SWEEP_DIV

    range_list_dict = {}
    config_list_dict = {}
    STEPS_1 = np.round(np.arange(RANGE1[0], RANGE1[1]+STEP1, STEP1), 4)
    STEPS_2 = np.round(np.arange(RANGE2[0], RANGE2[1]+STEP2, STEP2), 4)
    STEPS_X = np.round([0] + np.arange(RANGEX[0], RANGEX[1]-1e-3, STEP_SWEEP)[1:].tolist(), 4)
    LEN_1, LEN_2 = len(STEPS_1), len(STEPS_2)
    count = 0
    LEN_ALL = LEN_1*LEN_2
    gtimer.tic("test")
    for i_p1, pos1 in enumerate(STEPS_1):
        for i_p2, pos2 in enumerate(STEPS_2):
            count += 1
            with gtimer.block("loop"):
                pos0 = np.multiply(pos1, DIR1) + np.multiply(pos2, DIR2)
                key = tuple(np.round(pos0, 4))
                print("========= {} ({}/{})=========".format(key, count, LEN_ALL))
                T_be = SE3(R_be, pos0)
                T_bp = np.matmul(T_be, SE3(np.identity(3), (0.34,0,0.25)))
                gscene.create_safe(GEOTYPE.BOX, "sweep_surf", ROBOT_BASE, 
                                   dims = np.multiply(0.01, DIR1) + np.multiply(0.4, DIR2) + np.multiply(RANGEX[1]-RANGEX[0], DIRX), 
                                   center=T_bp[:3,3], rpy=Rot2rpy(T_bp[:3,:3]), 
                                   color=(0.8,0.0,0.0,0.5), display=True,  
                                   fixed=True, collision=True)
                gscene.update_markers_all()
                if VISUALIZE:
                    gscene.add_highlight_axis("hl", "tbe0", link_name=ROBOT_BASE, 
                                              center=T_be[:3,3], orientation_mat=T_be[:3,:3])
                
                p2p_list = []
                qpair_list = []
                T_be0 = np.copy(T_be)
                for i_try in range(N_try):
                    traj, success = mplan.planner.plan_py(
                        ROBOT_NAME, TIP_LINK, np.concatenate(T2xyzquat(T_be0)), ROBOT_BASE, HOME_POSE, 
                        timeout=TIMEOUT_MP)
                    if not success:
                        continue
                    for i_px, posx in enumerate(STEPS_X):
                        pos_sweep = np.multiply(posx, DIRX)
                        pos = pos0 + pos_sweep
                        T_be = SE3(R_be, pos)
                        if i_px > 0: # already done for 0
                            if abs(posx) < 1e-4: # already done for 0 at 0 index
                                continue
                            if VISUALIZE:
                                gscene.add_highlight_axis("hl", "tbe0", link_name=ROBOT_BASE, 
                                                          center=T_be[:3,3], orientation_mat=T_be[:3,:3])
                            traj, success = mplan.planner.plan_py(
                                ROBOT_NAME, TIP_LINK, np.concatenate(T2xyzquat(T_be)), ROBOT_BASE, HOME_POSE, 
                                timeout=TIMEOUT_MP)
                        if not success:
                            continue
                        Q0 = traj[-1]
                        if VISUALIZE:
                            gscene.show_pose(Q0)

                        pos_tar = np.multiply(RANGEX[0], DIRX)
                        dP_tar = pos_tar - pos_sweep
                        if VISUALIZE:
                            gscene.add_highlight_axis("hl", "tbe_n", link_name=ROBOT_BASE, 
                                                      center=T_be[:3,3]+dP_tar, orientation_mat=T_be[:3,:3])
                        Traj_down, success = get_sweep_traj(mplan, ee_point, dP_tar, Q0, DP=STEPX, ERROR_CUT=0.01, 
                                                            ref_link=ROBOT_BASE, VISUALIZE=VISUALIZE, VERBOSE=VERBOSE)
                        if len(Traj_down)==0:
                            continue
                        Qdown = Traj_down[-1]

                        pos_tar = np.multiply(RANGEX[1], DIRX)
                        dP_tar = pos_tar - pos_sweep
                        if VISUALIZE:
                            gscene.add_highlight_axis("hl", "tbe_p", link_name=ROBOT_BASE, 
                                                      center=T_be[:3,3]+dP_tar, orientation_mat=T_be[:3,:3])
                        Traj_up, success = get_sweep_traj(mplan, ee_point, dP_tar, Q0, DP=STEPX, ERROR_CUT=0.01, 
                                                          ref_link=ROBOT_BASE, VISUALIZE=VISUALIZE, VERBOSE=VERBOSE)
                        if len(Traj_up)==0:
                            continue
                        Qup = Traj_up[-1]

                        p0 = ee_point.get_tf(list2dict(Q0, gscene.joint_names))[SWEEP_AX_IDX,3]
                        pdown = ee_point.get_tf(list2dict(Qdown, gscene.joint_names))[SWEEP_AX_IDX,3]
                        pup = ee_point.get_tf(list2dict(Qup, gscene.joint_names))[SWEEP_AX_IDX,3]
                        p2p = (pdown, pup)
                        qpair = (Qdown, Q0, Qup)
                        if np.multiply(*np.sign(p2p))<0:
                            p2p_list.append(p2p)
                            qpair_list.append(qpair)

                range_list_dict[key] = np.array(p2p_list)
                config_list_dict[key] = np.array(qpair_list)
                if VISUALIZE:
                    gscene.clear_highlight()
        elapsed = gtimer.toc("test")/1000
        print(gtimer)
        print("=============================================")
        print("=============== {} / {} s===================".format(int(elapsed), int(elapsed/count*LEN_ALL)))

        print("=============================================")

    best_range_dict = {}
    for k,v in range_list_dict.items():
        if len(v)>0:
            i_max = np.argmax(v[:,1]-v[:,0])
            best_range_dict[k] = v[i_max]
        else:
            best_range_dict[k] = (0,0)
    np.save(DATASET_PATH, {"best_range_dict": best_range_dict, "range_list_dict": range_list_dict, "config_list_dict": config_list_dict})

========= (0.2, -0.5, 0.0) (1/1632)=========
========= (0.2, -0.48, 0.0) (2/1632)=========
========= (0.2, -0.46, 0.0) (3/1632)=========
========= (0.2, -0.44, 0.0) (4/1632)=========
========= (0.2, -0.42, 0.0) (5/1632)=========
========= (0.2, -0.4, 0.0) (6/1632)=========
========= (0.2, -0.38, 0.0) (7/1632)=========
========= (0.2, -0.36, 0.0) (8/1632)=========
========= (0.2, -0.34, 0.0) (9/1632)=========
========= (0.2, -0.32, 0.0) (10/1632)=========
========= (0.2, -0.3, 0.0) (11/1632)=========
========= (0.2, -0.28, 0.0) (12/1632)=========
========= (0.2, -0.26, 0.0) (13/1632)=========
========= (0.2, -0.24, 0.0) (14/1632)=========
========= (0.2, -0.22, 0.0) (15/1632)=========
========= (0.2, -0.2, 0.0) (16/1632)=========
========= (0.2, -0.18, 0.0) (17/1632)=========
========= (0.2, -0.16, 0.0) (18/1632)=========
========= (0.2, -0.14, 0.0) (19/1632)=========
========= (0.2, -0.12, 0.0) (20/1632)=========
========= (0.2, -0.1, 0.0) (21/1632)=========
========= (0.2, -0.08, 0.0)

========= (0.26, -0.4, 0.0) (159/1632)=========
========= (0.26, -0.38, 0.0) (160/1632)=========
========= (0.26, -0.36, 0.0) (161/1632)=========
========= (0.26, -0.34, 0.0) (162/1632)=========
========= (0.26, -0.32, 0.0) (163/1632)=========
========= (0.26, -0.3, 0.0) (164/1632)=========
========= (0.26, -0.28, 0.0) (165/1632)=========
========= (0.26, -0.26, 0.0) (166/1632)=========
========= (0.26, -0.24, 0.0) (167/1632)=========
========= (0.26, -0.22, 0.0) (168/1632)=========
========= (0.26, -0.2, 0.0) (169/1632)=========
========= (0.26, -0.18, 0.0) (170/1632)=========
========= (0.26, -0.16, 0.0) (171/1632)=========
========= (0.26, -0.14, 0.0) (172/1632)=========
========= (0.26, -0.12, 0.0) (173/1632)=========
========= (0.26, -0.1, 0.0) (174/1632)=========
========= (0.26, -0.08, 0.0) (175/1632)=========
========= (0.26, -0.06, 0.0) (176/1632)=========
========= (0.26, -0.04, 0.0) (177/1632)=========
========= (0.26, -0.02, 0.0) (178/1632)=========
========= (0.26, 0.0, 0.

In [ ]:
# np.save(DATASET_PATH, {"best_range_dict": best_range_dict, "range_list_dict": range_list_dict, "config_list_dict": config_list_dict})

## plot ranges

In [ ]:
# plt.figure(figsize=(15,15*LEN_2))
# for i_h, height_plot in enumerate(STEPS_2):
#     plt.subplot(LEN_2, 1, i_h+1)
#     plt.title(height_plot)
#     for k,v in range_list_dict.items():
#         if round(k[2], 4)==round(height_plot, 4):
#             for vv in v:
#                 plt.plot(vv, [k[0]+0.02, k[0]+0.02], '-o', linewidth=1)
#             plt.plot(best_range_dict[k], [k[0], k[0]], '-ks', linewidth=3)
#     plt.axis([-1,1,0,1])
#     plt.grid()

## Read and save min/max csv

In [ ]:
for sweep_dir in SweepDirections:
    DIRECTION = "-"+sweep_dir.name
    filename = SweepDirections.get_file_name(ROBOT_TYPE, sweep_dir)
    DATASET_PATH = os.path.join(DATASET_DIR, filename+".npy")
    dataset_read = np.reshape(np.load(DATASET_PATH, allow_pickle=True), (1,))[0]
    range_list_dict = dataset_read['range_list_dict']
    best_range_dict = dataset_read['best_range_dict']
    
    SWEEP_VEC = [0,1,0]
    min_points = []
    max_points = []
    for k,v in best_range_dict.items():
        min_points.append(tuple(np.add(k, np.multiply(v[0], SWEEP_VEC))))
        max_points.append(tuple(np.add(k, np.multiply(v[1], SWEEP_VEC))))
    min_points = sorted(min_points, key=lambda x: (x[2], x[0], x[1]))
    max_points = sorted(max_points, key=lambda x: (x[2], x[0], x[1]))
    np.savetxt(os.path.join(DATASET_DIR, filename+'-min.csv'), min_points, delimiter=",")
    np.savetxt(os.path.join(DATASET_DIR, filename+'-max.csv'), max_points, delimiter=",")

## Show saved ranges

In [ ]:
sweep_dir = SweepDirections.front
DIRECTION = "-"+sweep_dir.name
filename = SweepDirections.get_file_name(ROBOT_TYPE, sweep_dir)
DATASET_PATH = os.path.join(DATASET_DIR, filename+".npy")
sweep_path = os.path.join(DATASET_DIR, filename)
sweep_max = np.loadtxt(sweep_path + "-max.csv", delimiter=",")
sweep_min = np.loadtxt(sweep_path + "-min.csv", delimiter=",")
lines = [(p_min, p_max) for (p_min, p_max) in zip(sweep_min, sweep_max) 
         if p_min[2] == 0.66]

In [11]:
gscene.clear_highlight()
show_lines(gscene, lines, base_link=ROBOT_BASE, sweep_axis=SWEEP_AXIS)

NameError: name 'show_lines' is not defined

In [ ]:
# DATASET_PATH_BAK1 = os.path.join(DATASET_DIR+"_bak", ROBOT_TYPE.name+'-up.npy')
# DATASET_PATH_BAK2 = os.path.join(DATASET_DIR+"_bak2", ROBOT_TYPE.name+'-up.npy')
# dataset_read_bak1 = np.reshape(np.load(DATASET_PATH_BAK1, allow_pickle=True), (1,))[0]
# dataset_read_bak2 = np.reshape(np.load(DATASET_PATH_BAK2, allow_pickle=True), (1,))[0]
# for k, v in dataset_read_bak1.items():
#     v.update(dataset_read_bak2[k])
# np.save(DATASET_PATH, dataset_read_bak1)